In [ ]:
import sys
from newqt import MainWindow, QApplication
app = QApplication(sys.argv)     
window = MainWindow()
window.show()
app.exec()    

Error: Session cannot generate requests

In [1]:
from DataManipulation.indicators import DEMA,MYEMA,D1
from QTVisual.Plot import PlotPanel
from newqt import getOHLC_pickle
from time import time
t = time()
mdata = getOHLC_pickle("EURUSD_M_2010_2021.pkl")
data = mdata.resample('1H').agg({'Open': 'first', 
                    'High': 'max', 
                    'Low': 'min', 
                    'Close': 'last'}).dropna()
# 9min
demaperiod = 9

data["dema9_Hi"] = DEMA(data['High'],demaperiod)
data["dema9_Lo"] = DEMA(data['Low'],demaperiod)
data["dema9_Cl"] = DEMA(data['Open'],demaperiod)


print(data.columns)
print(time()-t)
t = time()
#plotpanels = [
#    PlotPanel(
#        data[["Open","High","Low","Close"]],
#    )
#]
#return plotpanels,data
import sys
from newqt import MainWindow, QApplication
app = QApplication(sys.argv)     
window = MainWindow(dfs = [data[["Open","High","Low","Close","dema9_Hi","dema9_Lo","dema9_Cl"]]])
window.show()
app.exec()   


Index(['Open', 'High', 'Low', 'Close', 'dema9_Hi', 'dema9_Lo', 'dema9_Cl'], dtype='object')
1.4156897068023682


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-sean'
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


0

In [1]:
x=1

In [1]:
import pyqtgraph as pg
from pyqtgraph import QtCore, QtGui

## Create a subclass of GraphicsObject.
## The only required methods are paint() and boundingRect() 
## (see QGraphicsItem documentation)
class CandlestickItem(pg.GraphicsObject):
    def __init__(self, data):
        pg.GraphicsObject.__init__(self)
        self.data = data  ## data must have fields: time, open, close, min, max
        self.generatePicture()
    
    def generatePicture(self):
        ## pre-computing a QPicture object allows paint() to run much more quickly, 
        ## rather than re-drawing the shapes every time.
        self.picture = QtGui.QPicture()
        p = QtGui.QPainter(self.picture)
        p.setPen(pg.mkPen('w'))
        w = (self.data[1][0] - self.data[0][0]) / 3.
        for (t, open, close, min, max) in self.data:
            p.drawLine(QtCore.QPointF(t, min), QtCore.QPointF(t, max))
            if open > close:
                p.setBrush(pg.mkBrush('r'))
            else:
                p.setBrush(pg.mkBrush('g'))
            p.drawRect(QtCore.QRectF(t-w, open, w*2, close-open))
        p.end()
    
    def paint(self, p, *args):
        p.drawPicture(0, 0, self.picture)
    
    def boundingRect(self):
        ## boundingRect _must_ indicate the entire area that will be drawn on
        ## or else we will get artifacts and possibly crashing.
        ## (in this case, QPicture does all the work of computing the bouning rect for us)
        return QtCore.QRectF(self.picture.boundingRect())

data = [  ## fields are (time, open, close, min, max).
    (1., 10, 13, 5, 15),
    (2., 13, 17, 9, 20),
    (3., 17, 14, 11, 23),
    (4., 14, 15, 5, 19),
    (5., 15, 9, 8, 22),
    (6., 9, 15, 8, 16),
]
item = CandlestickItem(data)
plt = pg.plot()
plt.addItem(item)
plt.setWindowTitle('pyqtgraph example: customGraphicsItem')

## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-sean'
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
